In [1]:
import sys
sys.path.append('../')
from collections import defaultdict
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from roverdata.db import DataWarehouse
from taxes_increase import utils

## Get LTV Data

In [2]:
LTV = pd.read_csv('../LTV.csv')

#### 180 Day LTV Data

In [7]:
LTV180 = LTV.loc[LTV.dates == 180]
new_180 = LTV180.loc[LTV180.new_repeat == 'new']
new_account_180 = LTV180.loc[LTV180.new_account == 'new']

#### 360 Day LTV Data

In [9]:
LTV360 = LTV.loc[LTV.dates == 360]
new_360 = LTV360.loc[LTV360.new_repeat == 'new']
new_account_360 = LTV360.loc[LTV360.new_account == 'new']

In [11]:
LTV360.gmv.sum()

134697450.9

In [13]:
LTV180.gmv.sum()/LTV360.gmv.sum()

0.56328184106712

In [14]:
LTV.head()

,requester_id,service,new_repeat,new_account,first_observed_stay_added,dates,stays,nrt,gmv
0,5726476,*,repeat,old,2018-08-25 02:40:16.000000,360,4,294.30,1400.0
1,7312458,dog-walking,new,old,2018-08-25 13:47:58.000000,180,25,355.05,1315.0
2,7192616,drop-in,new,old,2018-08-26 23:24:12.000000,360,21,125.01,463.0
3,1426949,dog-walking,repeat,old,2018-08-25 11:30:23.000000,30,5,34.02,126.0
4,5441756,*,repeat,old,2018-08-27 20:06:46.000000,180,7,37.80,140.0


In [18]:
LTV[LTV.new_repeat == 'new'][LTV.dates == 180].gmv.sum()/LTV[LTV.new_repeat == 'new'][LTV.dates == 360].gmv.sum()

/Users/avianapolsky/.local/share/virtualenvs/rover-ds-taxes-increase-5_PHDNU7/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0.5916358071435552

## Get Sample Size Data

In [19]:
samples = pd.read_csv('../retrans.csv')
samples_new = samples.loc[samples['new_repeat']=='new'][['service','num_owners']].groupby(['service']).sum()
samples_new_account = samples.loc[samples['new_account']=='new'][['service','num_owners']].set_index('service')

In [20]:
def project_30_day_samples(df, samples_col):
    """from df with column `service` and inputted samples column (containing 30 day samples)
    , project out 45 and 60 days"""
    df = df[samples_col].reset_index().rename(columns={samples_col:'30_day'})
    df['45_day'] = (df['30_day']*1.5).astype(int)
    df['60_day'] = (df['30_day']*2).astype(int)
    return df.set_index('service')


expected_xx_day_samples_new = project_30_day_samples(df = samples_new, samples_col = 'num_owners')
expected_xx_day_samples_new_account = project_30_day_samples(df = samples_new_account, samples_col = 'num_owners')

## Estimate Distribution of Sample Statistic

#### Estimate mean of means and standard error of means of sample statistic through simulation
http://blog.analytics-toolkit.com/2017/statistical-significance-non-binomial-metrics-revenue-time-site-pages-session-aov-rpu/

In [21]:
def sample_data(df, metric, samples, simulations):
    """
    given a dataframe `df` with a metric of interst column `metric`,
    select `samples` number of samples,
    repeat for `simulations` number of simulations
    
    Returns:
        meanOfMeans (float)
        seOfMeans (float)
    """
    metric_series = df[metric]
    means = np.array([metric_series.sample(n=samples, replace=True).mean() for i in range(simulations)])
    return means.mean(), means.std()

In [22]:
def run_sample_data(df, expected_xx_day_samples):
    """loop through the services, metrics, and sample sizes and estimate mean of means and std of means"""
    dist_stats_dict = {}
    
    for service in df.service.unique():
        df_service = df.loc[df['service'] == service]
        samples = expected_xx_day_samples.loc[service].to_dict()
        for days, samples in samples.items():
            for metric in ['gmv', 'nrt', 'stays']:
                mom, seom = sample_data(df_service, metric, samples, 1000)
                dist_stats_dict[(service, metric, days)] = (samples, mom, seom)  
                
    return pd.DataFrame.from_dict(dist_stats_dict, orient='index', columns=['samples','meanOfMeans','SEOfMeans'])

In [23]:
dist_stats_new = run_sample_data(
    df=new, 
    expected_xx_day_samples = expected_xx_day_samples_new
)
dist_stats_new_account = run_sample_data(
    df=new_account, 
    expected_xx_day_samples = expected_xx_day_samples_new_account
)

In [24]:
dist_stats_new_account

,samples,meanOfMeans,SEOfMeans
"(dog-walking, gmv, 30_day)",2459,232.916861,9.176955
"(dog-walking, nrt, 30_day)",2459,61.287989,2.253520
"(dog-walking, stays, 30_day)",2459,4.718914,0.161246
"(dog-walking, gmv, 45_day)",3688,233.158938,7.095506
"(dog-walking, nrt, 45_day)",3688,61.206509,1.938489
"(dog-walking, stays, 45_day)",3688,4.716653,0.130539
"(dog-walking, gmv, 60_day)",4918,232.716648,6.544851
"(dog-walking, nrt, 60_day)",4918,61.324864,1.630777
"(dog-walking, stays, 60_day)",4918,4.725431,0.111308
"(*, gmv, 30_day)",22334,139.962754,2.040004


In [ ]:
expected_xx_day_samples_new_account

## Find control-variant differneces that would be significant (alpha = 95%)

In [11]:
from scipy.stats import t

def independent_ttest(mean, se, samples, alpha, diff):
    mean1, mean2 = mean, mean*(1-diff)
    sem = se
    t_stat = (mean2 - mean1) / sem
    df = samples - 1
    cv = t.ppf(1.0 - alpha, df)
    pval = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
    return t_stat, df, cv, pval

In [12]:
def run_independent_ttest(dist_stats):
    """given estimates of the mean of means and se of means for each service/metric/sample size combo, 
    estimate minimum difference in means that would result in a significant result"""
    pval_dict = defaultdict(list)

    for index, row in dist_stats.iterrows():
        samples = row.samples*(2/3)
        mean = row.meanOfMeans
        se = row.SEOfMeans
        for diff in np.arange(.005, .15, .001):
            t_stat, df, cv, pval = independent_ttest(
                mean = mean, 
                se = se, 
                samples = samples*(1/2),
                alpha = .95,
                diff = diff
            )
            if pval < 0.05:
                pval_dict[(index)].append((diff, pval))
                break
                
    df = pd.DataFrame.from_dict(pval_dict, orient='index').reset_index()
    df[['service', 'metric', 'samples']] = pd.DataFrame(df['index'].tolist(), index=df.index) 
    df[['drop', 'pval']] = pd.DataFrame(df[0].tolist(), index=df.index) 
    df.drop(columns = ['index', 0], inplace=True)
    return df.pivot_table(index=['samples', 'service'], columns=['metric'], values = 'drop')


## New Customers % Drops for Significance

In [13]:
expected_xx_day_samples_new.columns.name = 'samples'
reshaped_samples_new = expected_xx_day_samples_new.stack().to_frame().sort_index().rename(columns = {0:'expected_samples'})

In [14]:
sig_drops_new = reshaped_samples_new.join(run_independent_ttest(dist_stats_new)*100, on = ['samples', 'service'])
sig_drops_new

expected_samples  gmv  nrt  stays
service     samples                                   
*           30_day              36150  2.2  2.2    2.3
            45_day              54225  1.8  1.8    1.9
            60_day              72300  1.6  1.6    1.8
dog-walking 30_day               3766  5.8  6.0    5.2
            45_day               5649  4.9  5.0    4.3
            60_day               7532  4.4  4.2    3.7
drop-in     30_day               6403  5.1  5.0    5.0
            45_day               9604  4.2  4.1    4.2
            60_day              12806  3.8  3.6    3.5
overnight   30_day              24635  2.8  2.7    2.8
            45_day              36952  2.2  2.2    2.1
            60_day              49270  2.0  1.9    1.9

## New Accounts % Drops for Significance

In [15]:
expected_xx_day_samples_new_account.columns.name = 'samples'
reshaped_samples_new_account = expected_xx_day_samples_new_account.stack().to_frame().sort_index().rename(columns = {0:'expected_samples'})

In [16]:
sig_drops_new_account = reshaped_samples_new_account.join(run_independent_ttest(dist_stats_new_account)*100, on = ['samples', 'service'])
sig_drops_new_account

expected_samples  gmv  nrt  stays
service     samples                                   
*           30_day              22334  3.0  2.9    3.1
            45_day              33501  2.4  2.5    2.6
            60_day              44668  2.2  2.1    2.2
dog-walking 30_day               2459  7.7  7.5    6.7
            45_day               3688  6.1  5.9    5.1
            60_day               4918  5.3  5.2    4.8
drop-in     30_day               4132  6.6  6.4    6.3
            45_day               6198  5.2  5.3    5.1
            60_day               8264  4.5  4.5    4.5
overnight   30_day              14797  3.7  3.6    3.5
            45_day              22195  3.1  2.9    3.1
            60_day              29594  2.6  2.5    2.6